### Import Library

In [1]:
# 이 중에서 안깔린 라이브러리가 있을 수 있음

import os
from typing import List, Dict, Tuple
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import seaborn as sns
import matplotlib.pyplot as plt
import pywt

### Load Data

In [2]:
path = "../../data/"

train = pd.read_csv(path+"train.csv").assign(_type="train")
test = pd.read_csv(path+"test.csv").assign(_type="test")
submission = pd.read_csv(path+"test.csv")
df = pd.concat([train, test], axis=0)

##### 원래 베이스라인 코드 부분

In [3]:
# # HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
# file_names: List[str] = [
#     f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
# ]

# # 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
# file_dict: Dict[str, pd.DataFrame] = {
#     f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
# }

# for _file_name, _df in tqdm(file_dict.items()):
#     # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
#     _rename_rule = {
#         col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
#         for col in _df.columns
#     }
#     _df = _df.rename(_rename_rule, axis=1)
#     df = df.merge(_df, on="ID", how="left")

##### 밑의 한 셀이 바꾼 부분 (이 바꾼 부분을 전제로 뒤의 코드가 실행됨.)

In [4]:
# HOURLY_로 시작하는 모든 csv 파일 불러오기
prefixes = ["HOURLY_NETWORK-DATA_",
            "HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD",
            "HOURLY_MARKET-DATA_COINBASE-PREMIUM-INDEX",
            "HOURLY_MARKET-DATA_FUNDING-RATES_ALL_EXCHANGE",
            "HOURLY_MARKET-DATA_LIQUIDATIONS_ALL_EXCHANGE_ALL_SYMBOL",
            "HOURLY_MARKET-DATA_OPEN-INTEREST_ALL_EXCHANGE_ALL_SYMBOL",
            "HOURLY_MARKET-DATA_TAKER-BUY-SELL-STATS_ALL_EXCHANGE"]

file_names = [
    f for f in os.listdir(path)
    if any(f.startswith(prefix) for prefix in prefixes) and f.endswith(".csv")
]
display(file_names)
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(path+f) for f in file_names
}


for _file_name, _df in tqdm(file_dict.items()):
    _rename_rule = {
        col: f"{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


['HOURLY_NETWORK-DATA_SUPPLY.csv',
 'HOURLY_MARKET-DATA_COINBASE-PREMIUM-INDEX.csv',
 'HOURLY_NETWORK-DATA_DIFFICULTY.csv',
 'HOURLY_NETWORK-DATA_ADDRESSES-COUNT.csv',
 'HOURLY_NETWORK-DATA_UTXO-COUNT.csv',
 'HOURLY_NETWORK-DATA_VELOCITY.csv',
 'HOURLY_MARKET-DATA_LIQUIDATIONS_ALL_EXCHANGE_ALL_SYMBOL.csv',
 'HOURLY_NETWORK-DATA_FEES.csv',
 'HOURLY_NETWORK-DATA_HASHRATE.csv',
 'HOURLY_MARKET-DATA_FUNDING-RATES_ALL_EXCHANGE.csv',
 'HOURLY_NETWORK-DATA_BLOCKREWARD.csv',
 'HOURLY_NETWORK-DATA_FEES-TRANSACTION.csv',
 'HOURLY_NETWORK-DATA_TRANSACTIONS-COUNT.csv',
 'HOURLY_MARKET-DATA_OPEN-INTEREST_ALL_EXCHANGE_ALL_SYMBOL.csv',
 'HOURLY_NETWORK-DATA_BLOCK-BYTES.csv',
 'HOURLY_NETWORK-DATA_TOKENS-TRANSFERRED.csv',
 'HOURLY_MARKET-DATA_PRICE-OHLCV_ALL_EXCHANGE_SPOT_BTC_USD.csv',
 'HOURLY_NETWORK-DATA_BLOCK-INTERVAL.csv',
 'HOURLY_NETWORK-DATA_BLOCK-COUNT.csv',
 'HOURLY_MARKET-DATA_TAKER-BUY-SELL-STATS_ALL_EXCHANGE.csv']

100%|██████████| 20/20 [00:00<00:00, 32.29it/s]


##### 결측치 (아거도 추가된 부분)

In [5]:
df_new = df.drop(["ID", "target", "_type"], axis = 1)

# 결측치 확인
missing_data = df_new.isnull().sum()

# 결측치 비율 확인
missing_ratio = df_new.isnull().mean()


display(missing_data.sort_values(ascending=False))
display(missing_ratio.sort_values(ascending=False))

volume                         2792
close                          2792
block_bytes                      54
funding_rates                    32
taker_buy_sell_ratio             31
taker_sell_ratio                 31
taker_buy_ratio                  31
taker_sell_volume                31
taker_buy_volume                 31
transactions_count_mean          28
supply_new                       28
supply_total                     28
fees_transaction_mean_usd        24
fees_block_mean                  24
fees_block_mean_usd              24
fees_reward_percent              24
difficulty                       24
tokens_transferred_mean          24
block_interval                   24
fees_transaction_mean            24
coinbase_premium_gap              6
coinbase_premium_index            6
transactions_count_total          4
open_interest                     4
block_count                       4
utxo_count                        1
addresses_count_sender            0
addresses_count_receiver    

volume                         0.241690
close                          0.241690
block_bytes                    0.004675
funding_rates                  0.002770
taker_buy_sell_ratio           0.002684
taker_sell_ratio               0.002684
taker_buy_ratio                0.002684
taker_sell_volume              0.002684
taker_buy_volume               0.002684
transactions_count_mean        0.002424
supply_new                     0.002424
supply_total                   0.002424
fees_transaction_mean_usd      0.002078
fees_block_mean                0.002078
fees_block_mean_usd            0.002078
fees_reward_percent            0.002078
difficulty                     0.002078
tokens_transferred_mean        0.002078
block_interval                 0.002078
fees_transaction_mean          0.002078
coinbase_premium_gap           0.000519
coinbase_premium_index         0.000519
transactions_count_total       0.000346
open_interest                  0.000346
block_count                    0.000346


In [6]:
# 1. 결측치 처리 - 앞/뒤 보간법 (forward/backward fill)

# df_ffill = df_new.fillna(method='ffill')  # 앞의 값을 사용하여 결측치 채움
# df_bfill = df_new.fillna(method='bfill')  # 뒤의 값을 사용하여 결측치 채움

# 2. 결측치 처리 - 선형 보간법 (linear interpolation)
df_interpolated = df_new.interpolate(method='linear')

# 3. 결측치 처리 - 평균값으로 대체
# df_mean_filled = df_new.fillna(eda_df.mean())

df.update(df_interpolated)

# 결측치 처리 후 확인
print(df_interpolated.isnull().sum())

supply_total                   0
supply_new                     0
coinbase_premium_gap           0
coinbase_premium_index         0
difficulty                     0
addresses_count_active         0
addresses_count_sender         0
addresses_count_receiver       0
utxo_count                     0
velocity_supply_total          0
long_liquidations              0
short_liquidations             0
long_liquidations_usd          0
short_liquidations_usd         0
fees_block_mean                0
fees_block_mean_usd            0
fees_total                     0
fees_total_usd                 0
fees_reward_percent            0
hashrate                       0
funding_rates                  0
blockreward                    0
blockreward_usd                0
fees_transaction_mean          0
fees_transaction_mean_usd      0
fees_transaction_median        0
fees_transaction_median_usd    0
transactions_count_total       0
transactions_count_mean        0
open_interest                  0
block_byte

### Feature engineering

만약 데이터를 로드할 때, (제공 받은)베이스라인 코드에서와 같이 거래소와 암호화폐 종류를 불러오도록 설정을 했다면

아래 코드는 작동하지 않음

거래소와 암호화폐 종류를 고려하지 않은 코드임을 유의.

(수정을 해야할 것 같음)

In [7]:
# 전체 독립변수에서 내가 사용할 독립변수 고르기


# 전체 feature
features = df.columns
# 내가 쓸 feature
fe = ['ID', 'target', '_type', 'funding_rates', 'open_interest',
       'taker_buy_volume', 'taker_sell_volume', 'taker_buy_ratio',
       'taker_sell_ratio', 'taker_buy_sell_ratio', 'difficulty',
       'transactions_count_total', 'transactions_count_mean', 'block_count',
       'fees_transaction_mean', 'fees_transaction_mean_usd',
       'fees_transaction_median', 'fees_transaction_median_usd',
       'fees_block_mean', 'fees_block_mean_usd', 'fees_total',
       'fees_total_usd', 'fees_reward_percent', 'hashrate', 'utxo_count',
       'tokens_transferred_total', 'tokens_transferred_mean',
       'tokens_transferred_median', 'block_interval', 'velocity_supply_total',
       'supply_total', 'supply_new', 'addresses_count_active',
       'addresses_count_sender', 'addresses_count_receiver',
       'blockreward', 'blockreward_usd', 'buy_sell_volume_ratio']

# new_features에는 내가 사용하지 않을 feature들이 담기고 나중에 drop됨. 때문에, 이름이 좀 잘못된 듯
new_features = [x for x in features if x not in fe]

In [8]:
# eda 에서 파악한 차이와 차이의 음수, 양수 여부를 새로운 피쳐로 생성
df = df.assign(
    liquidation_diff=df["long_liquidations"] - df["short_liquidations"],
    liquidation_usd_diff=df["long_liquidations_usd"] - df["short_liquidations_usd"],
    volume_diff=df["taker_buy_volume"] - df["taker_sell_volume"],
    liquidation_diffg=np.sign(df["long_liquidations"] - df["short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["long_liquidations_usd"] - df["short_liquidations_usd"]),
    volume_diffg=np.sign(df["taker_buy_volume"] - df["taker_sell_volume"]),
    buy_sell_volume_ratio=df["taker_buy_volume"] / (df["taker_sell_volume"] + 1),
)

# category, continuous 열을 따로 할당해둠
category_cols = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
conti_cols = [_ for _ in (df.columns)[3:] if _ not in ["ID", "target", "_type"]] + [
    "buy_sell_volume_ratio",
    "liquidation_diff",
    "liquidation_usd_diff",
    "volume_diff",
]

In [9]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict

# 지수 이동 평균
def EMA(df, col, span=2):
    return df[col].ewm(span=span).mean()

# Wavlet Transform
def WT(df, col, wavelet='db5', th=0.6):
    signal = df[col].values
    th = th*np.nanmax(signal)
    coef = pywt.wavedec(signal, wavelet, mode="per" )
    coef[1:] = (pywt.threshold(i, value=th, mode="soft" ) for i in coef[1:])
    reconstructed = pywt.waverec(coef, wavelet, mode="per" )
    return reconstructed

* 🔥 피처엔지니어링 함수 추가

In [10]:
# 날짜 관련 생성
def make_date_features(df: pd.DataFrame, date_column: str) -> Tuple[pd.DataFrame]:
    """
    입력된 데이터프레임의 특정 날짜 열을 기준으로 연도, 월, 주, 요일, 시간을 추출하여 새로운 피처로 추가.
    Args:
        df (pd.DataFrame): 날짜 컬럼을 포함하는 데이터프레임.
        date_column (str): 날짜 정보가 담긴 열 이름.
    Returns:
        pd.DataFrame: 날짜 관련 피처가 추가된 데이터프레임 반환.
    """
    df[date_column] = pd.to_datetime(df[date_column])
    df['year'] = df[date_column].dt.year  # 연도
    df['month'] = df[date_column].dt.month  # 월 
    df['week'] = df[date_column].dt.isocalendar().week  # 주
    df['day_of_week'] = df[date_column].dt.dayofweek  # 요일 
    df['hour'] = df[date_column].dt.hour  # 시간 
    date_columns = [date_column, 'year', 'month', 'week', 'day_of_week', 'hour']
    return df

In [11]:
# 변동성, 차분 피처 생성 함수 
def make_diff_change_feature(
    df: pd.DataFrame, 
    columns_list: List[str]
) -> pd.DataFrame:
    """
    주어진 변수들에 대한 변동성, 차분 피처를 생성하고, 결측값을 처리하는 함수.
    Args:
        df (pd.DataFrame): 데이터를 포함한 데이터프레임.
        columns_list (List[str]): 변동성과 차분 피처를 추가하고자 하는 열 이름 목록.
    Returns:
        pd.DataFrame: 변동성과 차분 피처가 추가된 데이터프레임.
    """  
    new_features = {}  # 새로 추가할 열을 저장할 딕셔너리
    for col in columns_list:
        if col in df.columns:
            pct_change_col = f'{col}_pct_change'
            diff_col = f'{col}_diff'
            new_features[pct_change_col] = df[col].pct_change(fill_method=None)
            new_features[diff_col] = df[col].diff()
        else:
            print(f"Error: Cannot find '{col}' column")
    new_features_df = pd.DataFrame(new_features, index=df.index) # 새 피처를 데이터프레임으로
    new_features_df = new_features_df.ffill().bfill()  # 결측값 처리 (고정: ffill, bfill)
    df = pd.concat([df, new_features_df], axis=1)  # 기존 데이터프레임에 추가
    return df

In [12]:
# 롱/숏 비율
def make_longshort_ratio_feature(
    df: pd.DataFrame, 
    long_col: str, 
    short_col: str
) -> pd.DataFrame:
    """
    롱/숏 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_col (str): 롱(liquidations) 데이터가 저장된 열 이름
        short_col (str): 숏(liquidations) 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 롱/숏 비율이 저장된 새로운 열이 추가된 데이터프레임
    """
    df['long_short_ratio'] = df[long_col] / (df[short_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [13]:
# 청산/거래량 비율
def make_liquidation_to_volume_ratio_feature(
    df: pd.DataFrame, 
    long_col: str, 
    short_col: str, 
    buy_volume_col: str, 
    sell_volume_col: str
) -> pd.DataFrame:
    """
    청산/거래량 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_col (str): 롱(liquidations) 데이터가 저장된 열 이름
        short_col (str): 숏(liquidations) 데이터가 저장된 열 이름
        buy_volume_col (str): 매수 거래량이 저장된 열 이름
        sell_volume_col (str): 매도 거래량이 저장된 열 이름
    Returns:
        pd.DataFrame: 청산/거래량 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['liquidation_to_volume_ratio'] = (
        (df[long_col] + df[short_col]) / 
        (df[buy_volume_col] + df[sell_volume_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    )
    return df

In [14]:
# 청산된 USD 롱/숏 비율
def make_liquidation_usd_ratio_feature(
    df: pd.DataFrame, 
    long_usd_col: str, 
    short_usd_col: str
) -> pd.DataFrame:
    """
    청산된 USD 롱/숏 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_usd_col (str): 롱(liquidations) USD 데이터가 저장된 열 이름
        short_usd_col (str): 숏(liquidations) USD 데이터가 저장된 열 이름 
    Returns:
        pd.DataFrame: 청산된 USD 롱/숏 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['liquidation_usd_ratio'] = df[long_usd_col] / (df[short_usd_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [15]:
# 펀딩 비율과 롱/숏 포지션 차이 곱
def make_funding_rate_position_change_feature(
    df: pd.DataFrame, 
    funding_rate_col: str, 
    long_liquidations_col: str, 
    short_liquidations_col: str
) -> pd.DataFrame:
    """
    펀딩 비율과 롱/숏 포지션 차이를 곱하여 포지션 변화를 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        funding_rate_col (str): 펀딩 비율 데이터가 저장된 열 이름
        long_liquidations_col (str): 롱 청산 데이터가 저장된 열 이름
        short_liquidations_col (str): 숏 청산 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 펀딩 비율에 따른 포지션 변화를 저장한 새로운 열이 추가된 데이터프레임
    """
    df['funding_rate_position_change'] = df[funding_rate_col] * (
        df[long_liquidations_col] - df[short_liquidations_col]
    )
    return df

In [16]:
# 프리미엄 갭과 프리미엄 인덱스의 차이
def make_premium_diff_feature(
    df: pd.DataFrame, 
    premium_gap_col: str, 
    premium_index_col: str
) -> pd.DataFrame:
    """
    프리미엄 갭과 프리미엄 인덱스의 차이를 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        premium_gap_col (str): 프리미엄 갭 데이터가 저장된 열 이름
        premium_index_col (str): 프리미엄 인덱스 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 프리미엄 갭과 프리미엄 인덱스의 차이를 저장한 새로운 열이 추가된 데이터프레임
    """
    df['premium_diff'] = df[premium_gap_col] - df[premium_index_col]
    return df


In [17]:
# 해시레이트와 난이도 간의 비율
def make_hashrate_to_difficulty_feature(
    df: pd.DataFrame, 
    hashrate_col: str, 
    difficulty_col: str
) -> pd.DataFrame:
    """
    해시레이트와 난이도 간의 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        hashrate_col (str): 해시레이트 데이터가 저장된 열 이름
        difficulty_col (str): 난이도 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 해시레이트와 난이도 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['hashrate_to_difficulty'] = df[hashrate_col] / (df[difficulty_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [18]:
# 공급 변화율
def make_supply_change_rate_feature(
    df: pd.DataFrame, 
    new_supply_col: str, 
    total_supply_col: str
) -> pd.DataFrame:
    """
    공급 변화율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        new_supply_col (str): 새로운 공급량 데이터가 저장된 열 이름
        total_supply_col (str): 총 공급량 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 공급 변화율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['supply_change_rate'] = df[new_supply_col] / (df[total_supply_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [19]:
# 적용 
df = make_date_features(df, 'ID') # 날짜 피처
df = make_diff_change_feature(df, ['open_interest']) # 변동성, 차분 피처 
df = make_longshort_ratio_feature(df, 'long_liquidations', 'short_liquidations') # 롱/숏 비율
df = make_liquidation_to_volume_ratio_feature(df, 'long_liquidations', 'short_liquidations', 'taker_buy_volume', 'taker_sell_volume') # 청산/거래량 비율
df = make_liquidation_usd_ratio_feature(df, 'long_liquidations_usd', 'short_liquidations_usd') # 청산된 USD 롱/숏 비율
df = make_funding_rate_position_change_feature(df, 'funding_rates', 'long_liquidations_usd', 'short_liquidations_usd') # 펀딩 비율과 롱/숏 포지션 차이 곱
df = make_premium_diff_feature(df, 'coinbase_premium_gap', 'coinbase_premium_index') # 프리미엄 갭과 프리미엄 인덱스의 차이
df = make_hashrate_to_difficulty_feature(df, 'hashrate', 'difficulty') # 해시레이트와 난이도 간의 비율
df = make_supply_change_rate_feature(df, 'supply_new', 'supply_total') # 공급 변화율

conti_cols = conti_cols + ['open_interest_pct_change', 'open_interest_diff', 'long_short_ratio', 'liquidation_to_volume_ratio', 'liquidation_usd_ratio', 'funding_rate_position_change', 'premium_diff', 'hashrate_to_difficulty', 'supply_change_rate']

In [20]:
# 위의 함수들을 실행한다.

# 모든 수치형 컬럼에 대한 지수이동평균 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"_moving_avg_7"] = EMA(df, c, 7)

# 모든 수치형 컬럼에 대한 Wavelet transform을 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"WT"] = WT(df, c)

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

/tmp/ipykernel_777029/2084871417.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"WT"] = WT(df, c)
/tmp/ipykernel_777029/2084871417.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"WT"] = WT(df, c)
/tmp/ipykernel_777029/2084871417.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  

In [21]:
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)
df = df.loc[:, ~df.columns.duplicated()]
df = df.drop(columns=new_features)

train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

### Model Training

#### lightGBM 학습

In [41]:
from lightgbm import LGBMClassifier

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# LightGBM 모델 생성 및 학습
lgb_model = LGBMClassifier(
    boosting_type="gbdt",
    objective="multiclass",
    num_class=4,
    num_leaves=50,
    learning_rate=0.05,
    n_estimators=30,
    random_state=42,
    verbose=-1
)

lgb_model.fit(
    x_train, 
    y_train,
    eval_set=[(x_valid, y_valid)],
    eval_metric='multi_logloss'
)

# 예측 확률
y_valid_pred_proba = lgb_model.predict_proba(x_valid)

# 클래스 레이블 예측
y_valid_pred_class = lgb_model.predict(x_valid)

# 성능 평가
accuracy_lgb = accuracy_score(y_valid, y_valid_pred_class)
auroc_lgb = roc_auc_score(y_valid, y_valid_pred_proba, multi_class="ovr")

print(f"LightGBM - acc: {accuracy_lgb}, auroc: {auroc_lgb}")

LightGBM - acc: 0.4366438356164384, auroc: 0.6439185931199092


#### XGBoost 학습

In [42]:
import xgboost as xgb
from sklearn.ensemble import VotingClassifier

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

xgb_params = {
    'objective': 'multi:softprob',
    'num_class': 4,
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 100,
    'random_state': 42
}

# XGBoost 모델 학습 및 성능 체크
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(x_train, y_train)

# XGBoost로 예측
y_valid_pred_xgb = xgb_model.predict_proba(x_valid)
y_valid_pred_class_xgb = xgb_model.predict(x_valid)

# XGBoost 성능 체크
accuracy_xgb = accuracy_score(y_valid, y_valid_pred_class_xgb)
auroc_xgb = roc_auc_score(y_valid, y_valid_pred_xgb, multi_class="ovr")

print(f"XGBoost - acc: {accuracy_xgb}, auroc: {auroc_xgb}")

XGBoost - acc: 0.4440639269406393, auroc: 0.648953293189703


#### Ensenble

In [43]:
ensemble_model = VotingClassifier(
    estimators=[
        ('lgb', lgb_model),
        ('xgb', xgb_model)
    ],
    voting='soft'
)

ensemble_model.fit(x_train, y_train)

y_valid_pred_ensemble = ensemble_model.predict_proba(x_valid)
y_valid_pred_class_ensemble = ensemble_model.predict(x_valid)

accuracy_ensemble = accuracy_score(y_valid, y_valid_pred_class_ensemble)
auroc_ensemble = roc_auc_score(y_valid, y_valid_pred_ensemble, multi_class="ovr")

print(f"Ensemble - acc: {accuracy_ensemble}, auroc: {auroc_ensemble}")

Ensemble - acc: 0.4394977168949772, auroc: 0.6523183340251395


In [45]:
# 성능 체크 후 전체 학습 데이터로 재학습
x_train_full = train_df.drop(["target", "ID"], axis=1)
y_train_full = train_df["target"].astype(int)

# LightGBM 전체 데이터 재학습
lgb_model_full = LGBMClassifier(
    boosting_type="gbdt",
    objective="multiclass",
    num_class=4,
    num_leaves=50,
    learning_rate=0.05,
    n_estimators=30,
    random_state=42,
    verbose=-1
)

lgb_model_full.fit(x_train_full, y_train_full)

# XGBoost 전체 데이터 재학습
xgb_model_full = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=4,
    learning_rate=0.05,
    max_depth=6,
    n_estimators=100,
    random_state=42
)

xgb_model_full.fit(x_train_full, y_train_full)

# 앙상블 모델 전체 데이터 재학습
ensemble_model_full = VotingClassifier(
    estimators=[
        ('lgb', lgb_model_full),
        ('xgb', xgb_model_full)
    ],
    voting='soft'
)

ensemble_model_full.fit(x_train_full, y_train_full)

VotingClassifier(estimators=[('lgb',
                              LGBMClassifier(learning_rate=0.05,
                                             n_estimators=30, num_class=4,
                                             num_leaves=50,
                                             objective='multiclass',
                                             random_state=42, verbose=-1)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categori...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_class=4, num_parallel_tree=None, ...))],
                 voting='soft')

In [48]:
y_valid_pred_ensemble = ensemble_model_full.predict_proba(x_valid)
y_valid_pred_class_ensemble = ensemble_model_full.predict(x_valid)


accuracy_ensemble = accuracy_score(y_valid, y_valid_pred_class_ensemble)
auroc_ensemble = roc_auc_score(y_valid, y_valid_pred_ensemble, multi_class="ovr")

print(f"Ensemble - acc: {accuracy_ensemble}, auroc: {auroc_ensemble}")

Ensemble - acc: 0.8807077625570776, auroc: 0.9943717564458966


### Inference

앙상블의 inference & save 과정.

In [ ]:
# 테스트 데이터에 대한 최종 예측
x_test = test_df.drop(["target", "ID"], axis=1)

y_test_pred_proba_ensemble = ensemble_model_full.predict_proba(x_test)
y_test_pred_class_ensemble = ensemble_model_full.predict(x_test)

# 제출 파일 생성
submission = submission.assign(target=y_test_pred_class_ensemble)
submission.to_csv("output.csv", index=False)